### 데이터 전처리
1. 필요없는 컬럼 삭제
2. 라벨을 가장 뒤로 보내기
3. 무한대값 처리
    * 무한대값 있는 컬럼 확인후 컬럼의 최대, 최소 값으로 대체
    * 무한대 값을 최대,최소 값으로 대체하는게 맞나...?
4. 이상값 처리 (윈저라이징1%)
5. 데이터 값 소수 둘째자리 반올림


In [11]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("결측치처리완료.csv")
df = df.drop(columns=['Unnamed: 0'])

In [13]:
columns_to_remove = [
    '회사명', '거래소코드','총자본순이익률', '매출액증가율',
    '자기자본순이익률', '매출액순이익률', '유동비율', '자기자본구성비율',
    '총자본회전률', '자기자본회전률', '총자본투자효율', '설비투자효율', '순이익증가율'
]

df = df.drop(columns=columns_to_remove)

label_column = df.pop('label')
df['label'] = label_column

In [14]:
def replace_infinity_except_column(df, exclude_column):
    for col in df.columns:
        if col != exclude_column and np.isinf(df[col]).any():
            # 양의 무한대 값을 해당 컬럼의 최대값으로 대체
            max_val = df[col][~np.isinf(df[col])].max()
            df[col].replace(np.inf, max_val, inplace=True)
            
            # 음의 무한대 값을 해당 컬럼의 최소값으로 대체
            min_val = df[col][~np.isinf(df[col])].min()
            df[col].replace(-np.inf, min_val, inplace=True)

# '회계년도' 컬럼을 제외하고 함수 실행
replace_infinity_except_column(df, '회계년도')

# 결과 출력
print(df)            

            회계년도  총자본순이익률_차이  총자본순이익률_비율  총자본순이익률_상위10개_차이  총자본순이익률_상위10개_변동성  \
0     2012-12-01      -10.20    1.827957           -46.709             46.709   
1     2013-12-01       16.30   -1.032953           -21.546             21.546   
2     2014-12-01       -0.24   -0.461538           -32.293             32.293   
3     2015-12-01       -6.65  -23.750000           -56.640             56.640   
4     2016-12-01        8.43   -1.323391           -40.578             40.578   
...          ...         ...         ...               ...                ...   
8622  2015-12-01       -2.40   -0.189573           -40.010             40.010   
8623  2016-12-01      -21.58   -2.103314           -53.958             53.958   
8624  2017-12-01        7.23   -0.638693           -81.108             81.108   
8625  2018-12-01       -5.05    1.234719           -45.142             45.142   
8626  2019-12-01       11.80   -1.291028           -42.441             42.441   

      총자본순이익률_산업평균_차이  총자본순

In [15]:
inf_columns = df.columns[df.isin([np.inf, -np.inf]).any()]
if not inf_columns.empty:
    print("무한대값 또는 -무한대값이 있는 컬럼:")
    for column in inf_columns:
        print(column)
else:
    print("무한대값 또는 -무한대값이 데이터프레임에 없습니다.")

print()

무한대값 또는 -무한대값이 데이터프레임에 없습니다.



In [16]:
def custom_winsorize(series, lower_quantile, upper_quantile):
    quantiles = series.quantile([lower_quantile, upper_quantile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# 윈저라이징 적용 (회계년도 및 라벨 컬럼 제외)
exclude_columns = ['회계년도', 'label']
for col in df.columns.drop(exclude_columns):
    df[col] = custom_winsorize(df[col], 0.01, 0.99)

# 결과 출력
print(df)

            회계년도  총자본순이익률_차이  총자본순이익률_비율  총자본순이익률_상위10개_차이  총자본순이익률_상위10개_변동성  \
0     2012-12-01      -10.20    1.827957           -46.709             46.709   
1     2013-12-01       16.30   -1.032953           -21.546             21.546   
2     2014-12-01       -0.24   -0.461538           -32.293             32.293   
3     2015-12-01       -6.65  -23.750000           -56.640             56.640   
4     2016-12-01        8.43   -1.323391           -40.578             40.578   
...          ...         ...         ...               ...                ...   
8622  2015-12-01       -2.40   -0.189573           -40.010             40.010   
8623  2016-12-01      -21.58   -2.103314           -53.958             53.958   
8624  2017-12-01        7.23   -0.638693           -81.108             81.108   
8625  2018-12-01       -5.05    1.234719           -45.142             45.142   
8626  2019-12-01       11.80   -1.291028           -42.441             42.441   

      총자본순이익률_산업평균_차이  총자본순

In [17]:
df = df.round(2)

In [18]:
df.to_csv("데이터전처리완료.csv")